Зчитуємо дані

In [66]:
import zipfile
with zipfile.ZipFile("homework.zip","r") as zip_ref:
    zip_ref.extractall()

In [67]:
import pandas as pd
import glob
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

In [68]:
data_walking = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/walking/*.csv"))
data_running = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/running/*.csv"))
data_idle = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/idle/*.csv"))
data_stairs = pd.concat(pd.read_csv(f) for f in glob.glob("/content/data/stairs/*.csv"))

In [69]:
data_walking['activity'] = 'walking'
data_running['activity'] = 'running'
data_idle['activity'] = 'idle'
data_stairs['activity'] = 'stairs'

data = pd.concat([data_walking, data_running, data_idle, data_stairs])

In [70]:
data.head(200)

,accelerometer_X,accelerometer_Y,accelerometer_Z,activity
0,0.574608,-14.470555,-3.586514,walking
1,-3.481169,-15.164873,-2.810793,walking
2,-4.673482,-10.927136,26.743233,walking
3,1.130063,-13.527240,-3.318364,walking
4,-11.755530,-14.767436,-16.730682,walking
...,...,...,...,...
15,-2.370260,-17.477673,-2.624045,walking
16,1.240197,-11.161768,-0.857124,walking
17,-5.090072,-19.632454,-3.327940,walking
18,-5.966350,-13.421895,-1.139640,walking


Тут я перевіряла унікальні значення для кожної осі - чи справді зчитано дані для всіх активностей

In [71]:
unique_activities_X = data.groupby('activity')['accelerometer_X'].unique()
unique_activities_Y = data.groupby('activity')['accelerometer_Y'].unique()
unique_activities_Z = data.groupby('activity')['accelerometer_Z'].unique()

print("Unique activities for accelerometer_X:")
print(unique_activities_X)

print("\nUnique activities for accelerometer_Y:")
print(unique_activities_Y)

print("\nUnique activities for accelerometer_Z:")
print(unique_activities_Z)


Unique activities for accelerometer_X:
activity
idle       [0.268151, 0.263362, 0.272939, 0.253785, 0.258...
running    [4.754885, -3.428497, 3.615245, 2.087744, 2.05...
stairs     [5.152323, 1.637634, -2.346318, -1.685518, -1....
walking    [0.574608, -3.4811690000000004, -4.673482, 1.1...
Name: accelerometer_X, dtype: object

Unique activities for accelerometer_Y:
activity
idle       [-0.0430959999999999, -0.033519, -0.004788, -0...
running    [1.264139, 6.090849, -7.302314999999999, -2.14...
stairs     [-11.702858, -22.720974, -3.270479, -7.580042,...
walking    [-14.470555, -15.164873000000002, -10.927136, ...
Name: accelerometer_Y, dtype: object

Unique activities for accelerometer_Z:
activity
idle       [9.763555, 9.758766, 9.768343, 9.773131, 9.749...
running    [-2.111686, -2.298433, -3.126827, -3.275268, 2...
stairs     [-2.27928, -10.491392, 1.063026, -1.623269, -3...
walking    [-3.586514, -2.810793, 26.743233, -3.318364, -...
Name: accelerometer_Z, dtype: object


Розрахунок деяких фічів - я не брала усі, які показував ментор, бо потім, коли він робив порівняльний аналіз впливу фічів на класифікацію, виявилося, що не всі є важливими. А ще тому, що не всі фічі вдається розрахувати з методом transform - багато з них він не підтримує, виникає помилка. Зокрема тому і range був розрахований по іншому

In [72]:
data['accelerometer_X_mean'] = data.groupby('activity')['accelerometer_X'].transform('mean')
data['accelerometer_X_std'] = data.groupby('activity')['accelerometer_X'].transform('std')
data['accelerometer_X_max'] = data.groupby('activity')['accelerometer_X'].transform('max')
data['accelerometer_X_min'] = data.groupby('activity')['accelerometer_X'].transform('min')
data['accelerometer_X_range'] = data['accelerometer_X_max'] - data['accelerometer_X_min']

data['accelerometer_Z_mean'] = data.groupby('activity')['accelerometer_Z'].transform('mean')
data['accelerometer_Z_std'] = data.groupby('activity')['accelerometer_Z'].transform('std')
data['accelerometer_Z_max'] = data.groupby('activity')['accelerometer_Z'].transform('max')
data['accelerometer_Z_min'] = data.groupby('activity')['accelerometer_Z'].transform('min')
data['accelerometer_Z_range'] = data['accelerometer_Z_max'] - data['accelerometer_Z_min']

data['accelerometer_Y_mean'] = data.groupby('activity')['accelerometer_Y'].transform('mean')
data['accelerometer_Y_std'] = data.groupby('activity')['accelerometer_Y'].transform('std')
data['accelerometer_Y_max'] = data.groupby('activity')['accelerometer_Y'].transform('max')
data['accelerometer_Y_min'] = data.groupby('activity')['accelerometer_Y'].transform('min')
data['accelerometer_Y_range'] = data['accelerometer_Y_max'] - data['accelerometer_Y_min']

In [73]:
data['accelerometer_mean_combined'] = data[['accelerometer_X_mean', 'accelerometer_Y_mean', 'accelerometer_Z_mean']].mean(axis=1)
data['accelerometer_std_combined'] = data[['accelerometer_X_std', 'accelerometer_Y_std', 'accelerometer_Z_std']].std(axis=1)
data['accelerometer_max_combined'] = data[['accelerometer_X_max', 'accelerometer_Y_max', 'accelerometer_Z_max']].max(axis=1)
data['accelerometer_min_combined'] = data[['accelerometer_X_min', 'accelerometer_Y_min', 'accelerometer_Z_min']].min(axis=1)
data['accelerometer_range_combined'] = data[['accelerometer_X_range', 'accelerometer_Y_range', 'accelerometer_Z_range']].max(axis=1)

Виділяємо ознаки та цільову змінну

In [74]:
features = data[['accelerometer_X_mean', 'accelerometer_X_std', 'accelerometer_X_max', 'accelerometer_X_min', 'accelerometer_X_range',
                 'accelerometer_Z_mean', 'accelerometer_Z_std', 'accelerometer_Z_max', 'accelerometer_Z_min', 'accelerometer_Z_range',
                 'accelerometer_Y_mean', 'accelerometer_Y_std', 'accelerometer_Y_max', 'accelerometer_Y_min', 'accelerometer_Y_range',
                 'accelerometer_mean_combined', 'accelerometer_std_combined', 'accelerometer_max_combined', 'accelerometer_min_combined', 'accelerometer_range_combined']]

target = data['activity']

Розбиваємо датасет на навчальний і тренувальний

In [75]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

Крос-валідація навчальних даних для методу опорних векторів

In [76]:
svm_model = SVC(C=1.0, class_weight='balanced', decision_function_shape='ovo', kernel='rbf', gamma='scale', probability=True)
svm_cv_scores = cross_val_score(svm_model, X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))
print("Cross-Validation Scores (SVM):", svm_cv_scores)
print("Mean Cross-Validation Score (SVM):", svm_cv_scores.mean())

Cross-Validation Scores (SVM): [1. 1. 1. 1. 1.]
Mean Cross-Validation Score (SVM): 1.0


Крос-валідація навчальних даних для випадкового лісу

In [77]:
rf_model = RandomForestClassifier(max_depth=5, class_weight='balanced')
rf_cv_scores = cross_val_score(rf_model, X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))
print("Cross-Validation Scores (Random Forest):", rf_cv_scores)
print("Mean Cross-Validation Score (Random Forest):", rf_cv_scores.mean())

Cross-Validation Scores (Random Forest): [1. 1. 1. 1. 1.]
Mean Cross-Validation Score (Random Forest): 1.0


Навчаємо моделі

In [78]:
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=5)

Тестуємо моделі, перевіряємо точність

In [79]:
svm_predict = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predict)

rf_predict = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predict)

print(svm_accuracy, rf_accuracy)

1.0 1.0


Мене турбує, що моделі показують надто високу точність - можливо оверфітінг та десь була допущена помилка? Саме для цього я додавала крос-валідацію та задавала максимально глибину для Рендом Форест, але все одно результат виходив 1.0 Тому навіть не знаю, як порівнювати моделі, якщо вони працюють з апріорі невірними даними (може була допущена помилка у розрахунку фічів, хоча здається всі ключові фічі враховані). Але якзщо вже порівнювати, то виходить, що метод опорних векторів та випадкового лісу однаково добре навчає модель класифікувати дані.